# Robot Motion Simulation

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import odeint

## Motion Models

### Damped Pendulum

The dynamics for a simple (damped) pendulum is defined by the second order equation:
\begin{equation}
\ddot{\theta} = -\frac{g}{l}\sin \theta - \gamma \dot{\theta},
\end{equation}
where $\theta$ is the angle of the pendulum with respect to the negative vertical axis, $g$ is the acceleration due to gravity, $l$ is the length of the pendulum, and $\gamma$ is a damping coefficient.
This second-order differential equation is equivalent to the system of first-order differential equations:
\begin{equation}
\dot{x}_0 = x_1, \quad \dot{x}_1 = -\frac{g}{l}\sin x_0 - \gamma x_1,
\end{equation}
where $x_0 = \theta$ and $x_1 = \dot{\theta}$.

In [ ]:
def dampled_pendulum(x, t, g=9.81, l=1., γ=1.):
    """
    Evaluate pendulum dynamics.

    Parameters
    ----------
    x : state vector, [θ, dθ_dt]
    t : time [s]
    g : acceleration due to gravity [m/s^2]
    l : pendulum length [m]
    γ : damping coefficient [1/s]

    Returns
    -------
    dx_dt : state vector derivative, [dθ_dt, d2θ_dt2]
    """
    θ, dθ = x
    ddθ = -(g/l)*np.sin(θ) - γ*dθ
    dx = np.array([dθ, ddθ])
    return dx

### Bicycle/Simple Car

A simple motion model for a bicycle/simple car is:
\begin{equation}
\dot{x} = v\cos\theta, \quad \dot{y} = v\sin\theta, \quad \dot{\theta} = \frac{v}{L}\tan\phi, 
\end{equation}
where $(x,y)$ is the position, $v$ is the speed control, $\theta$ is the heading angle, $L$ is the wheelbase length, and $\phi$ is the steering angle.
This is a controlled system with the control inputs being $u = [v, \phi]^\top$.
Here we can assume simple simple open-loop control with:
\begin{equation}
v = 1, \quad \phi = \sin(t).
\end{equation}

In [ ]:
def open_loop_bicycle(x, t, L=1.):
    """
    Evaluate bicycle dynamics with a simple open-loop control law.

    Parameters
    ----------
    x : state vector, [x, y, θ]
    t : time [s]
    L : wheelbase length [m]

    Returns
    -------
    dx_dt : state vector derivative, [̇dx_dt, dy_dt, dθ_dt]
    """
    # Open-loop controller definition
    v = 1 # constant speed
    ϕ = np.sin(t)

    ### Implement bicycle dynamics here
    return

# Simulating Dynamics

We can simulate motion models by solving the differential equations via numerical integration. In Python, the `SciPy` package offers a method for numerical integration called `odeint`. Run the code block below to simulated the damped pendulum using the function above.

In [ ]:
# Define constants
θ0 = np.deg2rad(30.) # initial angle
x0 = np.array([θ0, 0.])
T = 10. # simulation time
l = 1 # pendulum length
γ = 0.5 # damping coefficient

In [ ]:
# Use odeint to simulated the dynamics
t = np.linspace(0, T, num=100)
x_odeint = odeint(dampled_pendulum, x0, t, args=(9.81, 1, γ))
θ_odeint, dθ_odeint = x_odeint.T

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=150)
ax.plot(t, np.rad2deg(θ_odeint))
ax.set_title(r'$l = $' + f'{l:g}, ' + r'$\gamma = $' + f'{γ:g}')
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$\theta(t)$ [deg]')
plt.show()

## Integration Schemes

Behind the scenes, `odeint` is performing a numerical integration. There are many methods for numerical integration, with varying degrees of accuracy and computational intensity. One of the simplest integration schemes is the Euler method, which uses the update formula:
\begin{equation}
x(t + \Delta t) = x(t) + \Delta t f(x(t), t),
\end{equation}
to integrate the ordinary differential equation:
\begin{equation}
\dot{x} = f(x(t), t).
\end{equation}

In [ ]:
def euler(h, x, t, Δt, **kwargs):
    """
    Perform Euler integration step of ̇x = h(x,t)

    Parameters
    ----------
    h : function to integrate, h(x, t, **kwargs)
    x : state vector
    t : time
    Δt : time step to integrate over
    kwargs : key word args to pass to f

    Returns
    -------
    x[t + Δt] : state vector at t + Δt
    """
    return x + Δt*h(x, t, **kwargs)

In [ ]:
def integrate(h, x0, t, integrator, **kwargs):
    """
    Perform numerical integration of ̇x = h(x,t) from initial state x0.

    Parameters
    ----------
    h : function to integrate, h(x, t, **kwargs)
    x0 : initial state vector
    t : time vector [t0, t0 + Δt, ..., T] 
    integrator : integration method function, integrator (h, x, t, Δt, **kwargs) 
    kwargs : key word args to pass to f

    Returns
    -------
    x : state vector for times t
    """
    x = np.zeros((t.size, x0.size))
    x[0] = x0
    for i in range(t.size - 1):
        x[i + 1] = integrator(h, x[i], t[i], t[i+1] - t[i], **kwargs)
    return x

We can use the `euler` integration method above and compare the result to the `scipy.odeint` method. While the Euler method is very simple, it does not have very good numerical accuracy, especially if the step size is large.

In the script below, try playing with the $\Delta t$ parameter to see how the Euler method accuracy varies.

In [ ]:
Δt = 0.01
t = np.linspace(0, T, num=int(T/Δt))

# Run scipy.odeint
x_odeint = odeint(dampled_pendulum, x0, t, args=(9.81, l, γ))
θ_odeint, dθ_odeint = x_odeint.T

# Test Euler method
x_euler = integrate(dampled_pendulum, x0, t, euler, l=l, γ=γ)
θ_euler, dθ_euler = x_euler.T

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=150)
ax.plot(t, np.rad2deg(θ_odeint), label='odeint')
ax.plot(t, np.rad2deg(θ_euler), '--', label='euler')
ax.set_title(r'$l = $' + f'{l:g}, ' + r'$\gamma = $' + f'{γ:g}')
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$\theta(t)$ [deg]')
ax.legend()
plt.show()

## Exercises

### Exercise 1: Midpoint and RK4 Integration Schemes

The Midpoint method and the fourth-order Runge-Kutta (RK4) method are two alternatives to the Euler method with better accuracy.

Implement the Midpoint and RK4 methods in the functions below, and then compare the results for the pendulum dynamics.

In [ ]:
def midpoint(h, x, t, Δt, **kwargs):
    """
    Perform Midpoint integration step of ̇x = h(x,t)

    Parameters
    ----------
    f : function to integrate, h(x, t, **kwargs)
    x : state vector
    t : time
    Δt : time step to integrate over
    kwargs : key word args to pass to f

    Returns
    -------
    x[t + Δt] : state vector at t + Δt
    """
    
    ### Implement midpoint method here
    return 

In [ ]:
def rk4(h, x, t, Δt, **kwargs):
    """
    Perform fourth-order Runge-Kutta integration step of ̇x = h(x,t)

    Parameters
    ----------
    h : function to integrate, h(x, t, **kwargs)
    x : state vector
    t : time
    Δt : time step to integrate over
    kwargs : key word args to pass to h

    Returns
    -------
    x[t + Δt] : state vector at t + Δt
    """
    
    ### Implement RK4 method here
    return 

Run the script below to test the Midpoint and RK4 methods and compare to the Euler and `scipy.odeint` methods. Play around with the $\Delta t$ parameter to see how the timestep effects the results.

In [ ]:
Δt = 0.01
t = np.linspace(0, T, num=int(T/Δt))

# Run scipy.odeint
x_odeint = odeint(dampled_pendulum, x0, t, args=(9.81, l, γ))
θ_odeint, dθ_odeint = x_odeint.T

# Test Euler method
x_euler = integrate(dampled_pendulum, x0, t, euler, l=l, γ=γ)
θ_euler, dθ_euler = x_euler.T

# Test Midpoint method
x_midpoint = integrate(dampled_pendulum, x0, t, midpoint, l=l, γ=γ)
θ_midpoint, dθ_midpoint = x_midpoint.T

# Test RK4 method
x_rk4 = integrate(dampled_pendulum, x0, t, rk4, l=l, γ=γ)
θ_rk4, dθ_rk4 = x_rk4.T

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(5, 2), dpi=150)
ax.plot(t, np.rad2deg(θ_odeint), label='odeint')
ax.plot(t, np.rad2deg(θ_euler), '--', label='euler')
ax.plot(t, np.rad2deg(θ_midpoint), '--', label='midpoint')
ax.plot(t, np.rad2deg(θ_rk4), '--', label='RK4')
ax.set_title(r'$l = $' + f'{l:g}, ' + r'$\gamma = $' + f'{γ:g}')
ax.set_xlabel(r'$t$')
ax.set_ylabel(r'$\theta(t)$ [deg]')
ax.legend()
plt.show()

### Exercise 2: Simulate Bicycle Dynamics

Implement the `open_loop_bicycle` function to model the bicycle/simple car dynamics with the specified open-loop controller. Run the script below to simulate the dynamics using the RK4 integration method from Exercise 1.

In [ ]:
Δt = 0.01
t = np.linspace(0, T, num=int(T/Δt))

# Use RK4 method to simulate
x0 = np.array([0, 0, 0])
x_bike = integrate(open_loop_bicycle, x0, t, rk4, L=1)
pos_x_bike, pos_y_bike, θ_bike = x_bike.T

# Plot results
fig, ax = plt.subplots(1, 1, figsize=(5, 5), dpi=150)
ax.plot(pos_x_bike, pos_y_bike)
ax.set_xlabel(r'$x(t)$')
ax.set_ylabel(r'$y(t)$')
plt.show()